## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 셀레니움 사용 자동화

In [14]:
## 필요 라이브러리 사용 등록

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd

In [70]:
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [53]:
## 웹드레이버에서 소스 위치의 서울 링크 클릭 (생략)
path = 'div.loca_step1>div.loca_step1_cont>ul>li:nth-child(1)>a'
driver.find_element(By.CSS_SELECTOR,path).click()

In [71]:
## 웹드라이버에서 소스 위치 서울링크 클릭 (클래스 이름으로 검색)
name = "set_sido_cd_btn"
driver.find_element(By.CLASS_NAME,name).click()

In [52]:
# ## 웹 드라이버에서 소스 위치 부산링크 클릭(미사용)
# name = "set_sido_cd_btn"
# driver.find_elements(By.CLASS_NAME,name)[5].click()

In [72]:
# 웹 드라이버에서 소스 위치 전체지역 링크 클릭
all = 'set_gugun_cd_btn'
driver.find_elements(By.CLASS_NAME,all)[0].click()

In [73]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [74]:
starbuks_soup_list = soup.select('li.quickResultLstCon')

starbuks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

##### 서울 매장 리스트DF

In [98]:
sbSeoulStores = []
for item in starbuks_soup_list: # 613번 돌면서
    name = item.select('strong')[0].text.strip() #매장명
    lat = item['data-lat'].strip() # 위도
    lng = item['data-long'].strip() # 경도
    strongType = item.select('i')[0]['class'][0].split('_')[1] # 가게 종류(일반,리저브,DT,WT)
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    sbSeoulStores.append([name,lat,lng,strongType,address,tel])


In [101]:
dfStarBucksSeoul = pd.DataFrame(sbSeoulStores,columns=['매장명','위도','경도','매장 타입','주소','전화번호'])

In [103]:
dfStarBucksSeoul.tail()

,매장명,위도,경도,매장 타입,주소,전화번호
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232
612,중화역,37.60170912407773,127.07841136432036,general,서울특별시 중랑구 봉화산로 35 1층,1522-3232


In [104]:
# 결측치가 하나도 없음!
dfStarBucksSeoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     613 non-null    object
 1   위도      613 non-null    object
 2   경도      613 non-null    object
 3   매장 타입   613 non-null    object
 4   주소      613 non-null    object
 5   전화번호    613 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB


In [ ]:
dfStarBucksSeoul.to_excel('./data/starbukcs_seoul_list.xlsx',index=False)

##### 서울 열린데이터광장 OpenAPI 공공데이터 수집
주소 https://data.seoul.go.kr/

In [5]:
# 서울열린광장 API키
Seoul_Api_Key = '6459524e6164666736394656547a4c'

In [8]:
#서울 열린데이터 광장 호출 url
# url = f'https://openapi.seoul.go.kr:8088/{Seoul_Api_Key}/json/SdeTlSccoSigW/1/25'
# url

'https://openapi.seoul.go.kr:8088/6459524e6164666736394656547a4c/json/SdeTlSccoSigW/1/25'

In [ ]:
## 행정구역 데이터(위경도 + 구역)
- https://github.com/southkorea/seoul-maps